# Лабораторная работа 10
# Основы объектно-ориентированного программирования в Python. Проектирование классов

Шпак Андрей Валерьевич, 09.08.2022

# Задание 1

Проработайте материал лекции, представленный в файле $\textrm{2022_KM2_T10_OOP_design.html}$.

## Задание 1.1

Перенесите в свой блокнот Jupyter Notebook информационную структуру лекций, с указанием номера и темы раздела.

## Задание 1.2

Дополните каждый пример из лекционного материала аналогичным собственный примером и кратким комментарием к нему.

# Тема 10. Проектирование классов

## 10.1 Композиция

In [4]:
class Part:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return "Part with data: %s" % self.data
    
# определение класса на основе композиции

class Whole:
    def __init__(self, data1, data2):
        self.part1 = Part(data1) # внедренный объект
        self.part2 = Part(data2) # внедренный объект
        
    def __repr__(self):
        return "Whole contains %s and %s " % (self.part1, self.part2)
    
    
# ВОПРОС: в каком случае необходимо указывать скобки class Part() после имени класса? 

In [5]:
I1 = Whole(1, 2)
I1

Whole contains Part with data: 1 and Part with data: 2 

In [54]:
# мой пример
# получился список из котов
# под влиянием бинарного дерева поиска из предыдущей лабораторной
# пример для иллюстрации композиции - механизма связывания классов, не похожего на наследование
# класс реализует композицию, если экземпляр класса содержит внедренные объекты других классов
# объект-контейнер - экземпляр класса, который содержит внедренные объекты других классов
# класс Basket при создании экземпляра, инициализирует атрибут экземпляра ссылкой на экземпляр класса EmptyNode
# в дальнейшем при добавлении новых котов в корзину, будут создаваться экземпляры класса Cat

class Cat:
    def __init__(self, age, color, breed, next_cat_in_basket):
        self.age = age
        self.color = color
        self.breed = breed
        self.next_cat_in_basket = next_cat_in_basket
        
    def __repr__(self):
        return "I'm %s %s cat with %s color \n%s" % (self.breed, self.age, self.color, self.next_cat_in_basket)
    
    def add_new_cat(self, age, color, breed):
        self.next_cat_in_basket = self.next_cat_in_basket.add_new_cat(age, color, breed)
        return self
    
class Empty_basket:
    def __repr__(self):
        return "Empty basket. Let's add a new cat!"
    
    def add_new_cat(self, age, color, breed):
        return Cat(age, color, breed, self)
    
class Basket_with_cats:
    def __init__(self):
        self.whats_the_cat = Empty_basket()
    
    def __repr__(self):
        return repr(self.whats_the_cat)
    
    def add_new_cat(self, age, color, breed):
        self.whats_the_cat = self.whats_the_cat.add_new_cat(age, color, breed) 


In [47]:
basket = Basket_with_cats()
basket

Empty basket. Let's add a new cat!

In [48]:
basket.add_new_cat('kitty', 'british', 'blue')

In [49]:
basket

I'm blue kitty cat with british color 
Empty basket. Let's add a new cat!

In [50]:
basket.add_new_cat('old', 'scottish', 'yellow')

In [51]:
basket

I'm blue kitty cat with british color 
I'm yellow old cat with scottish color 
Empty basket. Let's add a new cat!

In [52]:
basket.add_new_cat('young', 'siberian', 'green')

In [53]:
basket

I'm blue kitty cat with british color 
I'm yellow old cat with scottish color 
I'm green young cat with siberian color 
Empty basket. Let's add a new cat!

## 10.2 Делегирование

In [57]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object
    
    def __getattr__(self, attrname):
        print('Trace: ', attrname)
        return getattr(self.wrapped, attrname)
    
x = Wrapper([1,2,3])
x.pop(); x.append(10); x.wrapped

# ВОПРОС: не понимаю, как тут перегружен __getattr__ и что он делает

Trace:  pop
Trace:  append


[1, 2, 10]

In [81]:
# мой пример
# просто перегрузил метод __init__ в делегирующем классе
# делегирование - разновидность композиции для создания класса-оболочки (Wrapper),
# когда экземпляр класса-оболочки содержит единственный внедренный объект (wrapped)
# большая часть интерфейса внедренного объекта сохраняется и дополняется

class Hello_wrapper:
    def __init__(self, object):
        if isinstance(object, str):
            self.wrapped = "hello " + object + " world"
        elif isinstance(object, list):       
            self.wrapped = ["hello"] + object + ["world"]
        elif isinstance(object, dict):
            object["hello"] = "world"
            self.wrapped = object
        elif isinstance(object, set):
            self.wrapped = object | {"hello", "world"}
        else:
            self.wrapped = "Also hello for tuples and nums"

In [70]:
hello1 = Hello_wrapper("123")
hello1.wrapped

'hello 123 world'

In [71]:
hello2 = Hello_wrapper([123])
hello2.wrapped

['hello', 123, 'world']

In [77]:
hello3 = Hello_wrapper({"abc":123})
hello3.wrapped

{'abc': 123, 'hello': 'world'}

In [78]:
hello4 = Hello_wrapper({123})
hello4.wrapped

{123, 'hello', 'world'}

In [79]:
hello5 = Hello_wrapper(123)
hello5.wrapped

'Also hello for tuples and nums'

In [80]:
hello5 = Hello_wrapper(123,)
hello5.wrapped

'Also hello for tuples and nums'

## 10.3 Абстрактный класс

In [93]:
class Super:
    def delegate(self):
        self.action()
        
    # метод-заглушка с выдачей сообщения об ошибке, если метод не переопределен в подклассах
    def action(self):
        assert False, "Action must be defined!"
        
class Provider(Super):
    def action(self):
        print("Action in Provider")

x = Provider()
x.delegate()

# ВОПРОС: в чем смысл заглушки абстрактного класса, если она работает только при вызове этого метода у экземпляра абстрактного класса?
# ВОПРОС: зачем в Provider'е в метод action был передан self?

Action in Provider


In [94]:
# три подкласса Dog, Cat, Cow класса Animal определяют неопределенный метод абстрактного класса
# подкласс Human нужен только чтобы посмотреть, что будет
# абстрактный класс - это класс, который содержит абстрактный метод
# в теле абстрактного метода вызывается метод, не определенный в дереве наследования: ни внутри самого класса, ни внутри его суперклассов
# вызов абстрактного метода для экземпляра, созданного на основе абстрактного класса, приведет к ошибке.



class Animal:
    def delegate(self):
        self.sound()
        
    def sound(self):
        assert False, "Sound is not defined"
        
class Dog(Super):
    def sound(self):
        print("bark")
        
class Cat(Super):
    def sound(self):
        print("meow")
        
class Cow(Super):
    def sound(self):
        print("muuuu")
        
class Human(Super):
    def speech(self):
        print("Hello world")

In [86]:
dog = Dog()
dog.sound()

bark


In [88]:
cat = Cat()
cat.sound()

meow


In [89]:
cow = Cow()
cow.sound()

muuuu


In [90]:
human = Human()
human.speech()
human.sound()

Hello world


AttributeError: 'Human' object has no attribute 'sound'

In [91]:
animal = Animal()
animal.sound()

AssertionError: Sound is not defined

In [95]:
import numbers

print(dir(numbers))

['ABCMeta', 'Complex', 'Integral', 'Number', 'Rational', 'Real', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'abstractmethod']


In [96]:
numbers.Number?

Init signature: numbers.Number()
Docstring:     
All numbers inherit from this class.

If you just want to check if an argument x is a number, without
caring what kind, use isinstance(x, Number).
File:           c:\users\andrey\appdata\local\programs\python\python310\lib\numbers.py
Type:           ABCMeta
Subclasses:     Complex


In [97]:
I1 = numbers.Complex()

TypeError: Can't instantiate abstract class Complex with abstract methods __abs__, __add__, __complex__, __eq__, __mul__, __neg__, __pos__, __pow__, __radd__, __rmul__, __rpow__, __rtruediv__, __truediv__, conjugate, imag, real

In [98]:
import collections.abc as abc

print(dir(abc))

['AsyncGenerator', 'AsyncIterable', 'AsyncIterator', 'Awaitable', 'ByteString', 'Callable', 'Collection', 'Container', 'Coroutine', 'Generator', 'Hashable', 'ItemsView', 'Iterable', 'Iterator', 'KeysView', 'Mapping', 'MappingView', 'MutableMapping', 'MutableSequence', 'MutableSet', 'Reversible', 'Sequence', 'Set', 'Sized', 'ValuesView', '_CallableGenericAlias', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [100]:
help(abc.Iterable)

Help on class Iterable in module collections.abc:

class Iterable(builtins.object)
 |  Methods defined here:
 |  
 |  __iter__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__ = GenericAlias(...) from abc.ABCMeta
 |      Represent a PEP 585 generic type
 |      
 |      E.g. for t = list[int], t.__origin__ is list and t.__args__ is (int,).
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should return True, False or NotImplemented.  If it returns
 |      NotImplemented, the normal algorithm is used.  Otherwise, it
 |      overrides the normal algorithm (and the outcome is cached).
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = froze

In [104]:
help(abc.Iterator)

Help on class Iterator in module collections.abc:

class Iterator(Iterable)
 |  Method resolution order:
 |      Iterator
 |      Iterable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __iter__(self)
 |  
 |  __next__(self)
 |      Return the next item from the iterator. When exhausted, raise StopIteration
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should return True, False or NotImplemented.  If it returns
 |      NotImplemented, the normal algorithm is used.  Otherwise, it
 |      overrides the normal algorithm (and the outcome is cached).
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = froze

## 10.4 Шаблон проектирования: синглтон

In [116]:
# сначала создается вспомогательный класс Singleton с единственным атрибутом класса _instance и 
# единственным методом __new__ для создания экземпляра класса Singleton

# метод __new__ контролирует, чтобы был создан только один экземпляр класса
# атрибут _instance хранит ссылку на единственный экземпляр класса

class Singleton:
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if not isinstance(cls._instance, cls):
            cls._instance = object.__new__(cls)
        return cls._instance
    
# ВОПРОС: не понимаю этот метод

In [115]:
# любой класс, который наследуется от Singleton
# экземпляр этого класса - единственный объект
# класс, на основе которого может быть создан только один экземпляр называется синглтон

class Simple(Singleton):
    def __init__(self, val1, val2):
        self.field1 = val1
        self.field2 = val2

In [114]:
# два экземпляра класса Simple

obj1 = Simple(2, 3); obj2 = Simple(3, 4)

In [113]:
# два экземпляра ссылаются на одну область памяти
obj1 is obj2

True

In [112]:
# значения атрибутов для двух экземпляров совпадают и определяются значениями экземпляра, созданного последним
obj1.field1, obj1.field2

(3, 4)

In [133]:
# класс, который наследуется от Singlton
# у этого класса только единственный экземпляр

class Cup(Singleton):
    def __init__(self, content):
        self.content = content
        
    def __repr__(self):
        return "I am a cup of " + self.content

In [134]:
# проверяю утыерждение

cup1 = Cup('coffie')
cup1

I am a cup of coffie

In [132]:
cup2 = Cup('tea')
cup1

I am a cup of tea

## 10.5 Классовые методы и статические методы

## 10.6 Особенности именования атрибутов и методов

## 10.7 Атрибут-свойство